In [ ]:
import os.path as osp
import pickle

import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.types as fot
import yaml
from fiftyone import ViewField as F

In [ ]:
fo.annotation_config.backends["cvat"]["segment_size"] = 300

In [ ]:
dataset_type = fot.FiftyOneDataset
config_file = "../../../config.yaml"

if osp.exists(config_file):
    print(f"Loading config from {config_file}")
    config = yaml.safe_load(config_file)
    dataset_dir = config["dataset_dir"]
    prediction_file = config["prediction_file"]
    categories = config["categories"]
    anno_key = config["anno_key"]
    prediction_field = config["prediction_field"]
    label_field = config["label_field"]
    temp_anno_field = config["temp_anno_field"]
else:
    dataset_dir = "/home/fkwong/datasets/82_truckcls/data/raw/truckcls-fiftyone"
    prediction_file = "/home/fkwong/workspace/srcs/public/grp_openmm/mmpretrain/work_dirs/test_outputs/efficientnet-b1_8xb32_truckcls/pred.pkl"
    categories = [
        "danger_vehicle",
        "dumper",
        "dumper6",
        "others",
        "pickup",
        "truck_box",
        "unknown",
    ]
    anno_key = "cvat_annotation"
    prediction_field = "efficientnet_b1"
    label_field = "ground_truth"  # classification
    temp_anno_field = "temp_annotation"

In [ ]:
dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    dataset_dir=dataset_dir,
)

In [ ]:
predictions = pickle.load(open(prediction_file, "rb"))
prediction_map = {}
for prediction in predictions:
    filepath = osp.abspath(prediction["img_path"])
    prediction_map[filepath] = prediction

In [ ]:
with dataset.save_context() as ctx:
    for sample in dataset:
        filepath = osp.abspath(sample.filepath)
        if filepath in prediction_map:
            prediction = prediction_map[filepath]
            idx = prediction["pred_label"].item()
            label = categories[idx]
            sample[prediction_field] = fo.Classification(
                label=label,
                confidence=prediction["pred_score"][idx].item(),
                logits=prediction["pred_score"].numpy().tolist(),
            )
            ctx.save(sample)

In [ ]:
prediction_view = dataset.match(~F(prediction_field).is_null())

In [ ]:
mistakenness_field = f"{prediction_field}_mistakenness"
fob.compute_mistakenness(
    prediction_view,
    prediction_field,
    "ground_truth",
    mistakenness_field=mistakenness_field,
)
anno_view = prediction_view.match((F(mistakenness_field) > 0.90)).sort_by(
    mistakenness_field,
    reverse=True,
)

In [ ]:
sess = fo.Session(anno_view, auto=False)

In [ ]:
anno_view = dataset.select(sess.selected)

In [ ]:
if len(anno_view) > 0:
    if anno_key in dataset.list_saved_views():
        print(f"{anno_key} view existed!")
    else:
        dataset.save_view(anno_key, anno_view)
else:
    print("No sample to be annotated.")

In [ ]:
if len(anno_view) > 0:
    sess = fo.Session(anno_view, auto=False)
else:
    print("No sample to be reviewed.")

In [ ]:
if len(anno_view) > 0:
    anno_results = anno_view.annotate(
        anno_key=anno_key,
        label_field=label_field,
        label_type="classification",
        classes=categories,
        launch_editor=False,
    )
else:
    print("No sample to be annotated.")

In [ ]:
anno_view = dataset.load_annotation_view(anno_key)
anno_view.load_annotations(
    anno_key,
    dest_field=temp_anno_field,
    cleanup=False,
)

In [ ]:
sess.view = anno_view

In [ ]:
anno_results = anno_view.load_annotation_results(anno_key)
for k, v in anno_results.get_status()[label_field].items():
    if v["status"] != "completed":
        print(f"Task-{k} is not completed yet, current status: {v['status']}")
        break
else:
    anno_view.load_annotations(anno_key, dest_field=label_field, cleanup=False)

In [ ]:
anno_results = dataset.load_annotation_results(
    anno_key,
    cache=False,
)

In [ ]:
dataset.delete_sample_fields(
    [
        temp_anno_field,
        mistakenness_field,
        prediction_field,
    ],
    error_level=1,
)

In [ ]:
dataset.export(
    export_dir=dataset_dir,
    dataset_type=dataset_type,
    export_media=True,
)

In [ ]:
anno_results.cleanup()
dataset.delete_annotation_run(anno_key)
dataset.delete_saved_view(anno_key)
dataset.list_saved_views()
dataset.list_annotation_runs()